<a href="https://colab.research.google.com/github/fermasia/infovis/blob/gh-pages/Android_Apps_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import altair as alt

In [ ]:
def prep_takeout(url,apps):
  '''
  Extraer del dump "Mi Actividad" de Google Takeout el uso de aplicaciones definidas por el nombre
  '''
  takeout = pd.read_json(url)
  takeout["time"] = pd.to_datetime(takeout["time"]).dt.tz_convert('America/Buenos_Aires') # Ajustar la TimeZone a Buenos Aires Argentina
  # Definir universo de Apps y Filtrar
  takeout = takeout[takeout["header"].isin(apps)]
  # Extraer fechas, horas y nombre del día
  takeout["date"] = pd.to_datetime(pd.to_datetime(takeout["time"],errors = 'coerce').dt.date)
  takeout["hour"] = pd.to_datetime(pd.to_datetime(takeout["time"],errors = 'coerce')).dt.hour
  takeout["day_name"] = pd.to_datetime(pd.to_datetime(takeout["time"],errors = 'coerce')).dt.day_name()
  takeout["day_name"] = takeout["day_name"].replace({"Monday": "1.Monday", "Tuesday": "2.Tuesday","Wednesday": "3.Wednesday","Thursday": "4.Thursday","Friday": "5.Friday","Saturday": "6.Saturday","Sunday": "7.Sunday"})
  takeout.rename(columns = {'header':'app'}, inplace = True) # Renombrar header
  takeout = takeout[["app","date","day_name","hour"]] # depurar columnas
  takeout = takeout[takeout.date > '2020-01-01']
  return takeout


In [ ]:
def crear_dataset(apps):
  '''
  Función para recorrer la lista de aplicaciones y crear un dataframe con fechas 
  y nombres de días a rellenar para cada aplicación entre 1-1-2021 y hoy
  '''
  df = pd.DataFrame(columns=["date","day_name","app"])
  for app in aplicaciones:
    today = pd.to_datetime('today').normalize()
    days = pd.DataFrame(pd.date_range('2021-01-01',today))
    days.columns = ["date"]
    days["day_name"] = pd.to_datetime(pd.to_datetime(days["date"],errors = 'coerce')).dt.day_name()
    days["day_name"] = days["day_name"].replace({"Monday": "1.Monday", "Tuesday": "2.Tuesday","Wednesday": "3.Wednesday","Thursday": "4.Thursday","Friday": "5.Friday","Saturday": "6.Saturday","Sunday": "7.Sunday"})
    days["app"] = app
    df = df.append(days, ignore_index = True)
  return df

In [ ]:
# Preparar Origen de Takeout
url = "https://raw.githubusercontent.com/fermasia/infovis/gh-pages/MiActividad.json"
aplicaciones = ['Twitter', 'Instagram', 'WhatsApp Messenger', 'Mercado Libre: compra fácil y rápido','Amazon Prime Video', 'Gmail','YouTube', 'Facebook','Spotify: Nueva música y podcasts para vos', 'Google Podcasts','Flow', 'Rappi: Delivery de Comida, Mercado y Farmacia','Cámara de Google','LinkedIn – App para Buscar Trabajo', 'Calendario de Google', 'Netflix','Google Maps','PedidosYa - Delivery Online']
takeout = prep_takeout(url,aplicaciones)

# Crear dataset base
df = crear_dataset(aplicaciones)

# Agrupar por fecha
takeout = takeout.groupby(["app","day_name","date"]).count().rename(columns={"hour":"cuenta"})
takeout = takeout.reset_index()

# Joinear con la tabla base
df = pd.merge(df,takeout,on=["date","app","day_name"],how="left")
df["cuenta"] = df["cuenta"].fillna(0)

#Calcular media
df_media_diaria = df.groupby(["app","day_name"]).mean().round(2).rename(columns={"cuenta":"promedio"})
df_media_diaria = df_media_diaria.reset_index()

In [ ]:
# Graficar 
alt.Chart(df_media_diaria).mark_bar().encode(
    x='day_name',
    y=alt.Y(
        'promedio',
        title='Usos Promedio',
        axis=alt.Axis(format='~s')
    ),
    facet=alt.Facet('app', columns=3),
).properties(
    title='Usos de Aplicaciones del Smartphone de Fernando',
    width=90,
    height=80
)

alt.Chart(...)